# Import

In [1]:
import os
import glob
import torch
import numpy as np

from dotenv import load_dotenv
from langchain_community.document_loaders import DirectoryLoader, Docx2txtLoader
from langchain_core.documents import Document
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
# from langchain_community.chat_models import ChatOllama
from langchain_openai import ChatOpenAI
from langchain_core.messages import ToolMessage

from langsmith import traceable
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage

load_dotenv()

/opt/miniconda3/envs/indiv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

# Embedding model

BAAI : https://huggingface.co/BAAI/bge-m3

intfloat : https://github.com/beir-cellar/beir?tab=readme-ov-file

In [2]:
embed_model = HuggingFaceEmbeddings(
    model_name="BAAI/bge-m3",   # Good at Thai
    model_kwargs={
        "device": "cuda" if torch.cuda.is_available() else "cpu"
    }
)

# embed_model = HuggingFaceEmbeddings(
#     model_name="intfloat/multilingual-e5-large",   # Good at Thai
#     model_kwargs={
#         "device": "cuda" if torch.cuda.is_available() else "cpu"
#     }
# )

print("Embedding model ready")


/var/folders/g2/p06t4qmx0ys102r1knn051hw0000gn/T/ipykernel_24406/1392817038.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embed_model = HuggingFaceEmbeddings(


Embedding model ready


# Load

## Train Docs

In [3]:
def load_train_docs(folder_path):
    docs = []

    files = sorted(glob.glob(os.path.join(folder_path, "*.docx")))

    for f in files:
        if os.path.basename(f).startswith("~$"):
            continue

        loader = Docx2txtLoader(f)
        doc = loader.load()[0]

        docs.append(
            Document(
                page_content=doc.page_content,
                metadata={"file": os.path.basename(f)}
            )
        )

    return docs

In [4]:
train_docs = load_train_docs("./dataset/train")

## Val Docs

In [5]:
def load_eval_docs(folder_path):
    docs = {}

    files = sorted(glob.glob(os.path.join(folder_path, "*.docx")))

    for f in files:
        if os.path.basename(f).startswith("~$"):
            continue

        loader = Docx2txtLoader(f)
        doc = loader.load()[0]

        docs[os.path.basename(f)] = doc.page_content

    return docs

In [6]:
val_docs = load_eval_docs("./dataset/val")

## Ground Truth Docs

In [7]:
def load_ground_truth(folder_path):
    gt = {}

    files = sorted(glob.glob(os.path.join(folder_path, "*.docx")))

    for f in files:
        if os.path.basename(f).startswith("~$"):
            continue

        loader = Docx2txtLoader(f)
        doc = loader.load()[0]

        gt[os.path.basename(f)] = doc.page_content

    return gt


In [8]:
ground_truth = load_ground_truth("./dataset/val_measure")

## Test Docs

In [9]:
def load_test_dataset(folder_path="./dataset/test"):

    test_inputs = {}
    test_ground_truth = {}

    files = sorted(glob.glob(os.path.join(folder_path, "*.docx")))

    for f in files:
        fname = os.path.basename(f)

        # Ignore temporary Word files
        if fname.startswith("~$"):
            continue

        loader = Docx2txtLoader(f)
        doc = loader.load()[0]
        text = doc.page_content.strip()

        if "มาตรการป้องกัน" not in text:
            print(f"[WARNING] ไม่พบคำว่า 'มาตรการป้องกัน' ในไฟล์ {fname}")
            continue

        parts = text.split("มาตรการ", 1)

        incident_text = parts[0].strip()
        measure_text = parts[1].strip()

        test_inputs[fname] = incident_text
        test_ground_truth[fname] = measure_text

    print(f"Loaded {len(test_inputs)} test cases")

    return test_inputs, test_ground_truth

In [10]:
test_inputs, test_ground_truth = load_test_dataset()

print(len(test_inputs))
print(list(test_inputs.keys())[:3])

Loaded 5 test cases
5
['รายงานสืบสวนอุบัติเหตุเชิงลึก_02_Feb19-PattayaChonburi.docx 17-49-22-743.docx', 'รายงานสืบสวนอุบัติเหตุเชิงลึก_05_280268_PickupRoadside.docx', 'รายงานสืบสวนอุบัติเหตุเชิงลึก_15_May03_Minibus.docx']


# Chunking Embed

In [13]:
# chunk_size, chunk_overlap = Hyper Params
splitter = RecursiveCharacterTextSplitter(
    chunk_size=250,
    chunk_overlap=50
)

chunks = splitter.split_documents(train_docs)
print(f"Total chunks: {len(chunks)}")


Total chunks: 528


# ChromaDB

## Drop all data in ChromaDB

In [ ]:
# import shutil

# RESET_DB = True

# if RESET_DB and os.path.exists("./chroma_db"):
#     shutil.rmtree("./chroma_db")

## Append embedding data

In [14]:
# Don't forget to create /chroma_db folder

persist_directory = "./chroma_db"

vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=embed_model,
    persist_directory=persist_directory
)
print("Embedding & Stored in ChromaDB successfully!")

Embedding & Stored in ChromaDB successfully!


# (Retrieval Evaluate)

In [15]:
retriever = vectorstore.as_retriever(search_kwargs={"k":3})

In [16]:
"""
    Test without Scores
"""

# docs = retriever.invoke("2 มกราคม 2568 เกิดอะไรขึ้น")
# docs = retriever.invoke("19 กุมภาพันธ์ 2568 เกิดอะไรขึ้น")
# docs = retriever.invoke("กิโลเมตรที่ 239+100")
docs = retriever.invoke("ชาย 28 ที่เป็นคนไทย")

# print(docs[0].page_content)
for i, d in enumerate(docs):
    print(f"\n--- Doc {i+1} ---")
    print("Source:", d.metadata)
    print(d.page_content[:300])



--- Doc 1 ---
Source: {'file': '2025-05-29 รายงานสืบสวนอุบัติเหตุเชิงลึก_RUTS-250315-07.docx'}
ผู้ขับขี่รถยนต์เก๋งเป็นเพศชาย อายุ 28 ปี สัญชาติไทย มีใบอนุญาตขับขี่รถยนต์ส่วนบุคคล ตรวจไม่พบปริมาณแอลกอฮอล์ในร่างกาย และไม่มีโรคประจำตัวที่อาจส่งผลต่อพฤติกรรมการขับขี่ ในวันเกิดเหตุ ผู้ขับขี่เดินทางกลับจากอำเภอขนอม มุ่งหน้าไปยังที่พักในอำเภอนบพิตำ

--- Doc 2 ---
Source: {'file': '2025-05-29 รายงานสืบสวนอุบัติเหตุเชิงลึก_RUTS-250315-07.docx'}
จากการสอบถามข้อมูลจากเจ้าหน้าที่ตำรวจและผู้ที่เกี่ยวข้อง ทราบว่า รถยนต์เก๋ง Mercedes-Benz สีขาว ขับขี่โดยชายไทย อายุ 28 ปี พร้อมผู้โดยสารอีก 2 ราย เดินทางกลับจากกิจกรรมคอนเสิร์ตในพื้นที่อำเภอขนอม ซึ่งห่างจากที่เกิดเหตุประมาณ 40 กิโลเมตร

--- Doc 3 ---
Source: {'file': '2025-05-29 รายงานสืบสวนอุบัติเหตุเชิงลึก_RUTS-250315-07.docx'}
ตารางที่ ‎3-1 ข้อมูลทางกายภาพ และข้อมูลการบาดเจ็บและเสียชีวิต

ตำแหน่ง

เพศ

สัญชาติ

อายุ

เข็มขัดนิรภัย

ระดับการบาดเจ็บ

D1

ชาย

ไทย

28

ใช้งาน

AIS 1 (แน่นหน้าอก)

11

ชาย

ไทย

26

ใช้งาน

AIS1 (บาดเจ็บเล็กน้อย)

21

ชา

In [17]:
"""
    Test with Scores
"""

search_msg = "คนไทย อายุ 28"
# search_msg = "2 มกราคม 2568 เกิดอะไรขึ้น"

docs_with_scores = vectorstore.similarity_search_with_score(
    search_msg,
    k=3
)


for doc, score in docs_with_scores:
    print("Score:", score)
    print("Source:", doc.metadata)
    print(doc.page_content[:200])
    print("-----")


Score: 0.9140981435775757
Source: {'file': '2025-05-29 รายงานสืบสวนอุบัติเหตุเชิงลึก_RUTS-250315-07.docx'}
ผู้ขับขี่รถยนต์เก๋งเป็นเพศชาย อายุ 28 ปี สัญชาติไทย มีใบอนุญาตขับขี่รถยนต์ส่วนบุคคล ตรวจไม่พบปริมาณแอลกอฮอล์ในร่างกาย และไม่มีโรคประจำตัวที่อาจส่งผลต่อพฤติกรรมการขับขี่ ในวันเกิดเหตุ ผู้ขับขี่เดินทางก
-----
Score: 0.9839334487915039
Source: {'file': '2025-06-03 รายงานสืบสวนอุบัติเหตุเชิงลึก_RUTS-250417-10.docx'}
และมีผู้โดยสารร่วมเดินทางมาด้วย 1 ราย เป็นหญิงไทย อายุ 23 ปี
-----
Score: 0.9969594478607178
Source: {'file': '2025-05-29 รายงานสืบสวนอุบัติเหตุเชิงลึก_RUTS-250315-07.docx'}
ผู้ขับขี่รถยนต์เก๋งเป็นชาย อายุ 28 ปี เดินทางพร้อมเพื่อนชายอีก 1 ราย หลังจากร่วมชมคอนเสิร์ตในพื้นที่อำเภอขนอม และกำลังมุ่งหน้ากลับบ้านพักในอำเภอนบพิตำ เมื่อเดินทางถึงบริเวณสี่แยกบ้านต้นเหรียง ซึ่งเป็น
-----


# LangSmith

Logging token and prompting

In [18]:
from langsmith import traceable

@traceable(name="LLM1-Tool-Selection")
def run_llm1(messages):
    return llm_with_tools.invoke(messages)


@traceable(name="Vector-Search")
def run_retrieval(query):
    return search_db.invoke(query)


@traceable(name="LLM2-Final-Reasoning")
def run_llm2(messages):
    return llm_final.invoke(messages)

In [19]:
@traceable(name="RAG-Agent-Pipeline")
def run_pipeline(question: str):

    messages = [HumanMessage(content=question)]

    response = run_llm1(messages)

    if not response.tool_calls:
        return response.content

    query = response.tool_calls[0]["args"]["query"]
    tool_result = run_retrieval(query)

    tool_message = ToolMessage(
        content=tool_result,
        tool_call_id=response.tool_calls[0]["id"]
    )

    final = run_llm2(messages + [response, tool_message])

    return final.content

# LangChain

## Blind LLM

In [20]:
# temperature = Hyper param
# LOW=Exactly in resource   HIGH=Creative

llm_blind = ChatOpenAI(
    model="qwen2.5",
    openai_api_key="ollama",
    openai_api_base="http://localhost:11434/v1",
    temperature=0
)

# llm_blind = ChatOllama(
#     model="qwen2.5",
#     temperature=0.1
# )

## Function Tools
Customize functions: the description & function name must be MEANINGFUL.

In [21]:
@tool
def search_db(query: str) -> str:
    """
    Search the accident investigation document database.

    Use this tool when the user asks about:
    - Specific accident reports
    - Dates of incidents
    - Causes of accidents
    - Number of injured or dead
    - Details inside official documents

    The query should be a concise search phrase in Thai,
    containing important keywords such as report ID,
    date, location, or accident type.

    Do NOT use this tool for general knowledge questions.
    """
    docs = retriever.invoke(query)
    return "\n\n".join([doc.page_content for doc in docs])


## LLM with Tool

In [22]:
llm_with_tools = llm_blind.bind_tools([search_db])

## Get Input from user

In [23]:
# user_input = input("Please describe the situation: ")

In [24]:
def build_extract_prompt(user_input: str) -> str:
    return f"""
<Instruction>
You are an expert accident investigation analyst.

Your task is to extract structured key information from the incident description provided in <Input>.

You MUST:
1. Extract only factual information explicitly stated in the input.
2. Do NOT infer, assume, or add any information not clearly mentioned.
3. Return the result strictly in valid JSON format.
4. If a field is not mentioned, return null.
5. Use concise but precise wording.
</Instruction>

<Context>
The structured output will be used to retrieve similar accident investigation cases 
from an internal database of .docx investigation reports.
Accuracy and structure are critical.
</Context>

<Input>
{user_input}
</Input>

<Output_Format>
Return ONLY valid JSON with the following structure:

{{
  "incident_type": "",
  "vehicles_involved": [],
  "environmental_conditions": "",
  "location_type": "",
  "casualties": {{
    "injured": 0,
    "fatalities": 0
  }},
  "immediate_causes": [],
  "contributing_factors": []
}}

</Output_Format>
"""

## Final LLM
Gathering HumanMessage, Generate Input and Tool Message together to be New Input for generating Final result

In [25]:
llm_final = ChatOpenAI(
    model="qwen2.5",
    openai_api_key="ollama",
    openai_api_base="http://localhost:11434/v1",
    temperature=0.3
)

In [26]:
def build_final_prompt(user_input: str,
                       structured_data: str,
                       retrieved_docs: list) -> str:
    """
    Build the final generation prompt for mitigation recommendation.
    """

    context_text = "\n\n".join(
        [doc.page_content for doc in retrieved_docs]
    )

    prompt = f"""
<Instruction>
You are a senior accident investigation and safety policy expert.

Your task is to generate appropriate mitigation measures for the incident described in <Input>, 
using ONLY the information contained in <Retrieved_Context>.

STRICT RULES:
1. You MUST base your reasoning strictly on the retrieved .docx investigation reports.
2. Do NOT use external knowledge.
3. Do NOT fabricate new policies.
4. If information is insufficient, state clearly: "Insufficient information in retrieved documents."
5. The answer must be logically derived from patterns, principles, or measures found in the retrieved documents.
6. The response must be structured and professional.

</Instruction>

<Context>
The structured extraction below was generated to retrieve similar cases.
Use it to understand the nature of the incident.
</Context>

<Structured_Extraction>
{structured_data}
</Structured_Extraction>

<Input>
{user_input}
</Input>

<Retrieved_Context>
{context_text}
</Retrieved_Context>

<Output_Format>

### 1. Key Risk Factors
- Bullet points summarizing major risks identified.

### 2. Recommended Mitigation Measures
- Concrete and actionable measures.
- Must align with principles found in retrieved documents.

### 3. Justification
- Explain how the measures are derived from the retrieved cases.

</Output_Format>
"""

    return prompt

# Define Pipeline

In [27]:
def run_pipeline(text_input):
    
    # Extract
    extract_prompt = build_extract_prompt(text_input)
    extracted = llm_with_tools.invoke(extract_prompt).content
    
    # Retrieve
    retrieved_docs = retriever.invoke(extracted)
    
    # Final generation
    final_prompt = build_final_prompt(
        text_input,
        extracted,        # structured_data
        retrieved_docs    # list of docs
    )
    
    final_answer = llm_final.invoke(final_prompt)
    
    return final_answer.content


# Evaluation

In [ ]:
# Cosine Similarity

def evaluate_similarity(pred, gt, embed_model):
    v1 = np.array(embed_model.embed_query(pred))
    v2 = np.array(embed_model.embed_query(gt))

    # Normalize vectors
    v1 = v1 / (np.linalg.norm(v1) + 1e-10)
    v2 = v2 / (np.linalg.norm(v2) + 1e-10)

    return float(np.dot(v1, v2))

## with Val Docs

In [31]:
predictions = {}

for fname, case_text in val_docs.items():
    print("Processing:", fname)
    prediction = run_pipeline(case_text)
    predictions[fname] = prediction

Processing: รายงานสืบสวนอุบัติเหตุเชิงลึก_KU-250305-57.docx
Processing: รายงานสืบสวนอุบัติเหตุเชิงลึก_KU-250312-67.docx
Processing: รายงานสืบสวนอุบัติเหตุเชิงลึก_KU-250322-71.docx
Processing: รายงานสืบสวนอุบัติเหตุเชิงลึก_NU-250109-01.docx
Processing: รายงานสืบสวนอุบัติเหตุเชิงลึก_NU-250608-01.docx


In [32]:
scores = []

for fname in predictions:
    if fname in ground_truth:
        score = evaluate_similarity(
            predictions[fname],
            ground_truth[fname],
            embed_model
        )

        print(fname, "=>", score)

        if not np.isnan(score):
            scores.append(score)

print('\n' + "#"*50 + '\n')
print("Scores list:", scores)
print("Length:", len(scores))
print("Average:", np.mean(scores))


รายงานสืบสวนอุบัติเหตุเชิงลึก_KU-250305-57.docx => 0.6069657875113526
รายงานสืบสวนอุบัติเหตุเชิงลึก_KU-250312-67.docx => 0.6492311301831859
รายงานสืบสวนอุบัติเหตุเชิงลึก_KU-250322-71.docx => 0.5936545595128556
รายงานสืบสวนอุบัติเหตุเชิงลึก_NU-250109-01.docx => 0.5801891977334518
รายงานสืบสวนอุบัติเหตุเชิงลึก_NU-250608-01.docx => 0.6424298266688377

##################################################

Scores list: [0.6069657875113526, 0.6492311301831859, 0.5936545595128556, 0.5801891977334518, 0.6424298266688377]
Length: 5
Average: 0.6144941003219367


In [37]:
'''
    Script: Test only 1 Val Docs
'''

fname = "รายงานสืบสวนอุบัติเหตุเชิงลึก_KU-250305-57.docx"

pred_text = predictions[fname]
gt_text = ground_truth[fname]

print("="*80)
print("FILE:", fname)
print("="*80)

print("\nPREDICTION:\n")
print(pred_text)

print("\nGROUND TRUTH:\n")
print(gt_text)

score = evaluate_similarity(pred_text, gt_text, embed_model)
print("\nSimilarity Score:", score)


FILE: รายงานสืบสวนอุบัติเหตุเชิงลึก_KU-250305-57.docx

PREDICTION:

### 1. Key Risk Factors

- **Driver Fatigue:** The driver of the motorcycle was experiencing symptoms of drowsiness and sleepiness.
- **Excessive Speeding:** The car was traveling at a speed significantly higher than the legal limit (over 120 km/h).
- **Lack of Defensive Driving:** The driver did not attempt to avoid or brake upon noticing the obstacle.

### 2. Recommended Mitigation Measures

1. **Driver Fatigue Management:**
   - Implement mandatory rest periods for drivers, especially during long journeys.
   - Provide educational programs on recognizing and managing fatigue symptoms among drivers.
   
2. **Speed Control Measures:**
   - Install speed monitoring devices in vehicles to enforce speed limits effectively.
   - Increase the number of speed cameras along high-risk routes.

3. **Enhanced Defensive Driving Training:**
   - Develop and promote defensive driving courses for all road users, focusing on recogni

## with Test Docs

In [33]:
test_predictions = {}

for fname, case_text in test_inputs.items():
    print("Processing:", fname)
    prediction = run_pipeline(case_text)
    test_predictions[fname] = prediction


Processing: รายงานสืบสวนอุบัติเหตุเชิงลึก_02_Feb19-PattayaChonburi.docx 17-49-22-743.docx
Processing: รายงานสืบสวนอุบัติเหตุเชิงลึก_05_280268_PickupRoadside.docx
Processing: รายงานสืบสวนอุบัติเหตุเชิงลึก_15_May03_Minibus.docx
Processing: รายงานสืบสวนอุบัติเหตุเชิงลึก_PSU-250512-01.docx
Processing: รายงานสืบสวนอุบัติเหตุเชิงลึก_PSU-250620-01.docx


In [34]:
test_scores = []

for fname in test_predictions:
    if fname in test_ground_truth:
        score = evaluate_similarity(
            test_predictions[fname],
            test_ground_truth[fname],
            embed_model
        )

        print(fname, "=>", score)

        if not np.isnan(score):
            test_scores.append(score)

print("\n" + "#"*50 + "\n")
print("Test Scores:", test_scores)
print("Length:", len(test_scores))
print("Average Test Score:", np.mean(test_scores))

รายงานสืบสวนอุบัติเหตุเชิงลึก_02_Feb19-PattayaChonburi.docx 17-49-22-743.docx => 0.7002609991918045
รายงานสืบสวนอุบัติเหตุเชิงลึก_05_280268_PickupRoadside.docx => 0.6509822222282975
รายงานสืบสวนอุบัติเหตุเชิงลึก_15_May03_Minibus.docx => 0.7013152272504422
รายงานสืบสวนอุบัติเหตุเชิงลึก_PSU-250512-01.docx => 0.66280872943394
รายงานสืบสวนอุบัติเหตุเชิงลึก_PSU-250620-01.docx => 0.6682452456495097

##################################################

Test Scores: [0.7002609991918045, 0.6509822222282975, 0.7013152272504422, 0.66280872943394, 0.6682452456495097]
Length: 5
Average Test Score: 0.6767224847507988
